# 베스트셀러 예측 ML 모델

## 분석 개요
- **목표**: 도서의 주간 베스트셀러 진입 여부 및 판매 점수 예측
- **접근법**: 2단계 모델 (Two-Stage)
  - 1단계 (분류): 베스트셀러 진입 여부 (0/1)
  - 2단계 (회귀): 진입 시 판매 점수 (1~20)
- **데이터 분할**: 시간순 Split (2025년 1~9월 train, 10~12월 test)
- **Validation**: 2026년 1월 데이터 (추후 적용)

---
## 1. 라이브러리 및 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report,
    mean_squared_error, mean_absolute_error, r2_score
)
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 로드 완료")

In [ ]:
# 데이터 로드
df = pd.read_csv('./books_ml_dataset.csv')
df['ymw'] = df['ymw'].astype(str)

print(f"✅ 데이터 로드 완료")
print(f"   - 행 수: {len(df):,}개")
print(f"   - 컬럼 수: {len(df.columns)}개")
print(f"\n📋 컬럼 목록:")
print(df.columns.tolist())

---
## 2. EDA (탐색적 데이터 분석)

In [ ]:
# Y 변수 분포 확인
print("📊 Y 변수 (y_sales_score) 분포")
print("="*50)
print(df['y_sales_score'].describe())

print(f"\n베스트셀러 진입 (score > 0): {(df['y_sales_score'] > 0).sum():,}개 ({(df['y_sales_score'] > 0).mean()*100:.1f}%)")
print(f"베스트셀러 미진입 (score = 0): {(df['y_sales_score'] == 0).sum():,}개 ({(df['y_sales_score'] == 0).mean()*100:.1f}%)")

In [ ]:
# Y 변수 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 전체 분포
axes[0].hist(df['y_sales_score'], bins=21, edgecolor='black', alpha=0.7)
axes[0].set_title('y_sales_score 전체 분포')
axes[0].set_xlabel('판매 점수')
axes[0].set_ylabel('빈도')

# 베스트셀러 진입 데이터만
bestseller_scores = df[df['y_sales_score'] > 0]['y_sales_score']
axes[1].hist(bestseller_scores, bins=20, edgecolor='black', alpha=0.7, color='green')
axes[1].set_title('베스트셀러 진입 데이터만 (score > 0)')
axes[1].set_xlabel('판매 점수')
axes[1].set_ylabel('빈도')

plt.tight_layout()
plt.show()

In [ ]:
# 주차별 분포 확인
df['year_month'] = df['ymw'].str[:6]  # YYYYMM 추출

monthly_stats = df.groupby('year_month').agg({
    'y_sales_score': ['count', 'mean', lambda x: (x > 0).sum()]
}).round(2)
monthly_stats.columns = ['총 행수', '평균 점수', '베스트셀러 진입 수']

print("📊 월별 데이터 분포")
print(monthly_stats)

In [ ]:
# 피처 상관관계 (Y와의 상관)
feature_cols = [c for c in df.columns if c not in ['product_code', 'ymw', 'y_sales_score', 'year_month']]

correlations = df[feature_cols + ['y_sales_score']].corr()['y_sales_score'].drop('y_sales_score').sort_values(ascending=False)

print("📊 Y 변수와의 상관계수 (상위/하위 10개)")
print("="*50)
print("[상위 10개]")
print(correlations.head(10))
print("\n[하위 10개]")
print(correlations.tail(10))

In [ ]:
# 상관계수 시각화
plt.figure(figsize=(12, 8))
correlations.plot(kind='barh', color=['green' if x > 0 else 'red' for x in correlations])
plt.title('피처별 Y 변수(y_sales_score)와의 상관계수')
plt.xlabel('상관계수')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

---
## 3. 데이터 전처리 및 분할

### 결측치 현황 및 원인

**결측치 발생 컬럼**: `category_1_predicted` ~ `category_10_predicted` (656개 행)

**결측치 발생 주차**:
- 2025035 (3월 5주차)
- 2025055 (5월 5주차)
- 2025085 (8월 5주차)
- 2025115 (11월 5주차)

**원인**: ymw 형식 불일치
```
베스트셀러 데이터:  2025035, 2025055, 2025085, 2025115 (3월, 5월, 8월, 11월 5주차)
뉴스/Prophet 데이터: 2025045, 2025075, 2025095, 2025125 (4월, 7월, 9월, 12월 5주차)
```
→ 두 데이터 소스의 **월별 주차 계산 방식이 다름**
→ Prophet 예측값이 해당 주차에 존재하지 않아 결측치 발생

**처리 방법**: 결측치를 0으로 대체 (해당 주차에는 Prophet 기반 예측 피처 미반영)

In [ ]:
# 피처 및 타겟 분리
feature_cols = [c for c in df.columns if c not in ['product_code', 'ymw', 'y_sales_score', 'year_month']]

X = df[feature_cols].copy()
y_score = df['y_sales_score'].copy()  # 회귀용
y_binary = (df['y_sales_score'] > 0).astype(int)  # 분류용 (진입 여부)

print(f"✅ 피처/타겟 분리 완료")
print(f"   - 피처 수: {len(feature_cols)}개")
print(f"   - 분류 타겟 (y_binary): 0={y_binary.value_counts()[0]:,}, 1={y_binary.value_counts()[1]:,}")

In [ ]:
# 시간순 분할 (2025년 1~9월: train, 10~12월: test)
# ymw 형식: YYYYMMW (예: 2025091 = 2025년 9월 1주차)

train_mask = df['ymw'].str[:6].astype(int) <= 202509  # 9월까지
test_mask = df['ymw'].str[:6].astype(int) >= 202510   # 10월부터

X_train, X_test = X[train_mask], X[test_mask]
y_train_binary, y_test_binary = y_binary[train_mask], y_binary[test_mask]
y_train_score, y_test_score = y_score[train_mask], y_score[test_mask]

print(f"✅ 시간순 분할 완료")
print(f"   - Train (2025년 1~9월): {len(X_train):,}개")
print(f"   - Test (2025년 10~12월): {len(X_test):,}개")
print(f"\n   - Train 베스트셀러 진입률: {y_train_binary.mean()*100:.1f}%")
print(f"   - Test 베스트셀러 진입률: {y_test_binary.mean()*100:.1f}%")

In [ ]:
# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# DataFrame으로 변환 (피처 이름 유지)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test.index)

print("✅ 스케일링 완료")

---
## 4. 1단계: 분류 모델 (베스트셀러 진입 예측)

불균형 데이터 처리를 위해 `class_weight='balanced'` 적용

In [ ]:
# 분류 모델 정의
clf_models = {
    'Logistic Regression': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(scale_pos_weight=len(y_train_binary[y_train_binary==0])/len(y_train_binary[y_train_binary==1]), 
                            random_state=42, eval_metric='logloss'),
    'LightGBM': LGBMClassifier(class_weight='balanced', random_state=42, verbose=-1)
}

print(f"📌 분류 모델 {len(clf_models)}개 정의 완료")
print(f"   - 불균형 처리: class_weight='balanced' 또는 scale_pos_weight 적용")

In [ ]:
# 분류 모델 학습 및 평가
clf_results = []

for name, model in clf_models.items():
    print(f"\n{'='*50}")
    print(f"[{name}] 학습 중...")
    
    # 학습
    model.fit(X_train_scaled, y_train_binary)
    
    # 예측
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # 평가
    acc = accuracy_score(y_test_binary, y_pred)
    prec = precision_score(y_test_binary, y_pred)
    rec = recall_score(y_test_binary, y_pred)
    f1 = f1_score(y_test_binary, y_pred)
    auc = roc_auc_score(y_test_binary, y_pred_proba)
    
    clf_results.append({
        'Model': name,
        'Accuracy': round(acc, 4),
        'Precision': round(prec, 4),
        'Recall': round(rec, 4),
        'F1-Score': round(f1, 4),
        'AUC-ROC': round(auc, 4)
    })
    
    print(f"  Accuracy: {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall: {rec:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  AUC-ROC: {auc:.4f}")

clf_results_df = pd.DataFrame(clf_results).sort_values('F1-Score', ascending=False)
print("\n" + "="*60)
print("📊 분류 모델 성능 비교 (F1-Score 기준 정렬)")
print("="*60)
print(clf_results_df.to_string(index=False))

In [ ]:
# 최고 성능 모델의 Confusion Matrix
best_clf_name = clf_results_df.iloc[0]['Model']
best_clf_model = clf_models[best_clf_name]
y_pred_best = best_clf_model.predict(X_test_scaled)

cm = confusion_matrix(y_test_binary, y_pred_best)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['미진입(0)', '진입(1)'], 
            yticklabels=['미진입(0)', '진입(1)'])
plt.title(f'Confusion Matrix - {best_clf_name}')
plt.xlabel('예측')
plt.ylabel('실제')
plt.tight_layout()
plt.show()

print(f"\n📋 Classification Report - {best_clf_name}")
print(classification_report(y_test_binary, y_pred_best, target_names=['미진입', '진입']))

In [ ]:
# 분류 모델 Feature Importance
if hasattr(best_clf_model, 'feature_importances_'):
    importance = best_clf_model.feature_importances_
elif hasattr(best_clf_model, 'coef_'):
    importance = np.abs(best_clf_model.coef_[0])
else:
    importance = None

if importance is not None:
    feat_imp_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 8))
    sns.barplot(data=feat_imp_df.head(20), x='importance', y='feature', palette='viridis')
    plt.title(f'Feature Importance (분류) - {best_clf_name}')
    plt.xlabel('중요도')
    plt.tight_layout()
    plt.show()
    
    print("\n📊 상위 10개 피처")
    print(feat_imp_df.head(10).to_string(index=False))

---
## 5. 2단계: 회귀 모델 (판매 점수 예측)

베스트셀러 진입 데이터만 사용 (y_sales_score > 0)

In [ ]:
# 베스트셀러 진입 데이터만 필터링
train_bs_mask = (y_train_score > 0)
test_bs_mask = (y_test_score > 0)

X_train_bs = X_train_scaled[train_bs_mask]
X_test_bs = X_test_scaled[test_bs_mask]
y_train_bs = y_train_score[train_bs_mask]
y_test_bs = y_test_score[test_bs_mask]

print(f"✅ 베스트셀러 진입 데이터 필터링 완료")
print(f"   - Train: {len(X_train_bs):,}개")
print(f"   - Test: {len(X_test_bs):,}개")
print(f"\n   - Train 평균 점수: {y_train_bs.mean():.2f}")
print(f"   - Test 평균 점수: {y_test_bs.mean():.2f}")

In [ ]:
# 회귀 모델 정의
reg_models = {
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=0.1, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
    'LightGBM': LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
}

print(f"📌 회귀 모델 {len(reg_models)}개 정의 완료")

In [ ]:
# 회귀 모델 학습 및 평가
reg_results = []

for name, model in reg_models.items():
    print(f"\n{'='*50}")
    print(f"[{name}] 학습 중...")
    
    # 학습
    model.fit(X_train_bs, y_train_bs)
    
    # 예측
    y_pred = model.predict(X_test_bs)
    
    # 평가
    mse = mean_squared_error(y_test_bs, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_bs, y_pred)
    r2 = r2_score(y_test_bs, y_pred)
    
    reg_results.append({
        'Model': name,
        'RMSE': round(rmse, 4),
        'MAE': round(mae, 4),
        'R²': round(r2, 4)
    })
    
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R²: {r2:.4f}")

reg_results_df = pd.DataFrame(reg_results).sort_values('R²', ascending=False)
print("\n" + "="*60)
print("📊 회귀 모델 성능 비교 (R² 기준 정렬)")
print("="*60)
print(reg_results_df.to_string(index=False))

In [ ]:
# 최고 성능 회귀 모델의 예측 vs 실제 시각화
best_reg_name = reg_results_df.iloc[0]['Model']
best_reg_model = reg_models[best_reg_name]
y_pred_best_reg = best_reg_model.predict(X_test_bs)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 산점도
axes[0].scatter(y_test_bs, y_pred_best_reg, alpha=0.5)
axes[0].plot([0, 21], [0, 21], 'r--', label='Perfect Fit')
axes[0].set_xlabel('실제 점수')
axes[0].set_ylabel('예측 점수')
axes[0].set_title(f'예측 vs 실제 - {best_reg_name}')
axes[0].legend()

# 잔차 분포
residuals = y_test_bs - y_pred_best_reg
axes[1].hist(residuals, bins=20, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='red', linestyle='--')
axes[1].set_xlabel('잔차 (실제 - 예측)')
axes[1].set_ylabel('빈도')
axes[1].set_title(f'잔차 분포 - {best_reg_name}')

plt.tight_layout()
plt.show()

In [ ]:
# 회귀 모델 Feature Importance
if hasattr(best_reg_model, 'feature_importances_'):
    importance_reg = best_reg_model.feature_importances_
elif hasattr(best_reg_model, 'coef_'):
    importance_reg = np.abs(best_reg_model.coef_)
else:
    importance_reg = None

if importance_reg is not None:
    feat_imp_reg_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': importance_reg
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 8))
    sns.barplot(data=feat_imp_reg_df.head(20), x='importance', y='feature', palette='magma')
    plt.title(f'Feature Importance (회귀) - {best_reg_name}')
    plt.xlabel('중요도')
    plt.tight_layout()
    plt.show()
    
    print("\n📊 상위 10개 피처 (회귀)")
    print(feat_imp_reg_df.head(10).to_string(index=False))

---
## 6. Feature Selection

In [ ]:
from sklearn.feature_selection import RFE, SelectFromModel

# 분류 모델 기반 Feature Selection
print("📌 Feature Selection (분류 모델 기준)")
print("="*50)

# LightGBM 기반 SelectFromModel
selector_clf = SelectFromModel(LGBMClassifier(class_weight='balanced', random_state=42, verbose=-1), 
                               threshold='median')
selector_clf.fit(X_train_scaled, y_train_binary)

selected_features_clf = np.array(feature_cols)[selector_clf.get_support()]
print(f"\n선택된 피처 수: {len(selected_features_clf)}개")
print(f"선택된 피처: {list(selected_features_clf)}")

In [ ]:
# 선택된 피처로 분류 모델 재학습
X_train_selected = X_train_scaled[selected_features_clf]
X_test_selected = X_test_scaled[selected_features_clf]

print("📌 선택된 피처로 분류 모델 재학습")
print("="*50)

clf_selected_results = []

for name, model in clf_models.items():
    # 새 모델 인스턴스 생성
    if name == 'Logistic Regression':
        m = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
    elif name == 'Random Forest':
        m = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)
    elif name == 'XGBoost':
        m = XGBClassifier(scale_pos_weight=len(y_train_binary[y_train_binary==0])/len(y_train_binary[y_train_binary==1]), 
                        random_state=42, eval_metric='logloss')
    else:
        m = LGBMClassifier(class_weight='balanced', random_state=42, verbose=-1)
    
    m.fit(X_train_selected, y_train_binary)
    y_pred = m.predict(X_test_selected)
    y_pred_proba = m.predict_proba(X_test_selected)[:, 1]
    
    clf_selected_results.append({
        'Model': name,
        'F1-Score': round(f1_score(y_test_binary, y_pred), 4),
        'AUC-ROC': round(roc_auc_score(y_test_binary, y_pred_proba), 4)
    })

clf_selected_df = pd.DataFrame(clf_selected_results).sort_values('F1-Score', ascending=False)
print("\n📊 Feature Selection 후 분류 성능")
print(clf_selected_df.to_string(index=False))

# 비교
print("\n📊 Feature Selection 전후 비교 (F1-Score)")
comparison = pd.merge(
    clf_results_df[['Model', 'F1-Score']].rename(columns={'F1-Score': 'Before'}),
    clf_selected_df[['Model', 'F1-Score']].rename(columns={'F1-Score': 'After'}),
    on='Model'
)
comparison['Diff'] = comparison['After'] - comparison['Before']
print(comparison.to_string(index=False))

---
## 7. 결과 요약 및 해석

In [ ]:
print("="*70)
print("📊 최종 결과 요약")
print("="*70)

print("\n[1단계: 분류 모델 - 베스트셀러 진입 예측]")
print("-"*50)
print(f"최고 성능 모델: {clf_results_df.iloc[0]['Model']}")
print(f"  - F1-Score: {clf_results_df.iloc[0]['F1-Score']}")
print(f"  - AUC-ROC: {clf_results_df.iloc[0]['AUC-ROC']}")
print(f"  - Precision: {clf_results_df.iloc[0]['Precision']}")
print(f"  - Recall: {clf_results_df.iloc[0]['Recall']}")

print("\n[2단계: 회귀 모델 - 판매 점수 예측]")
print("-"*50)
print(f"최고 성능 모델: {reg_results_df.iloc[0]['Model']}")
print(f"  - R²: {reg_results_df.iloc[0]['R²']}")
print(f"  - RMSE: {reg_results_df.iloc[0]['RMSE']}")
print(f"  - MAE: {reg_results_df.iloc[0]['MAE']}")

print("\n[Feature Selection 결과]")
print("-"*50)
print(f"전체 피처 수: {len(feature_cols)}개")
print(f"선택된 피처 수: {len(selected_features_clf)}개")
print(f"선택된 피처: {list(selected_features_clf)}")

In [ ]:
# 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 분류 모델 비교
ax1 = axes[0]
x = range(len(clf_results_df))
width = 0.35
ax1.bar([i - width/2 for i in x], clf_results_df['F1-Score'], width, label='F1-Score', color='steelblue')
ax1.bar([i + width/2 for i in x], clf_results_df['AUC-ROC'], width, label='AUC-ROC', color='orange')
ax1.set_xticks(x)
ax1.set_xticklabels(clf_results_df['Model'], rotation=45, ha='right')
ax1.set_ylabel('Score')
ax1.set_title('분류 모델 성능 비교')
ax1.legend()
ax1.set_ylim(0, 1)

# 회귀 모델 비교
ax2 = axes[1]
colors = ['green' if r > 0 else 'red' for r in reg_results_df['R²']]
ax2.barh(reg_results_df['Model'], reg_results_df['R²'], color=colors)
ax2.set_xlabel('R² Score')
ax2.set_title('회귀 모델 성능 비교 (R²)')
ax2.axvline(x=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

---
## 8. 모델 저장

In [ ]:
import joblib

# 최고 성능 모델 저장
joblib.dump(best_clf_model, './best_classifier.pkl')
joblib.dump(best_reg_model, './best_regressor.pkl')
joblib.dump(scaler, './scaler.pkl')

# 결과 CSV 저장
clf_results_df.to_csv('./classification_results.csv', index=False)
reg_results_df.to_csv('./regression_results.csv', index=False)

print("✅ 모델 및 결과 저장 완료")
print("   - best_classifier.pkl")
print("   - best_regressor.pkl")
print("   - scaler.pkl")
print("   - classification_results.csv")
print("   - regression_results.csv")

---
## 9. 인사이트 및 해석

### 분류 모델 해석
- **목표**: 특정 도서가 특정 주차에 베스트셀러 Top 20에 진입할지 예측
- **불균형 처리**: class_weight='balanced' 적용으로 소수 클래스(진입) 학습 강화
- **주요 지표**: F1-Score, AUC-ROC (불균형 데이터에서 Accuracy보다 신뢰성 높음)

### 회귀 모델 해석
- **목표**: 베스트셀러 진입 시 예상 순위 점수 (1~20점) 예측
- **데이터**: 실제 베스트셀러 진입 데이터만 사용
- **주요 지표**: R², RMSE, MAE

### Feature Importance 인사이트
- 중요 피처 분석을 통해 어떤 요인이 베스트셀러 진입/순위에 영향을 미치는지 파악
- 바이럴 지수, 카테고리 관련성, 거시경제 지표의 상대적 중요도 비교

### 비즈니스 활용
1. **출판사**: 신간 출시 시점 최적화, 마케팅 대상 도서 선정
2. **서점**: 재고 관리, 프로모션 기획
3. **저자/에이전시**: 출간 타이밍 전략

---
## 10. 다음 단계 (TODO)

1. **2026년 1월 Validation 데이터 적용**
   - 별도 수집 후 모델 검증
   - 실제 운영 환경 성능 확인

2. **하이퍼파라미터 튜닝**
   - GridSearchCV 또는 Optuna 활용
   - 최적 파라미터 탐색

3. **앙상블 모델 구성**
   - 분류 + 회귀 파이프라인
   - Stacking, Voting 등

4. **Prophet 피처 추가**
   - Walk-Forward Prophet 예측값 피처로 추가
   - 시차 효과 반영